#### 02-Vector Search Homework

In [1]:
import json
import numpy as np

from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

p:\Projects\LLM-ZoomCamp\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Q1. Embedding the query

Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?

In [24]:
embedding_model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')


text = 'I just discovered the course. Can I join now?'


embeddings = list(embedding_model.embed([text]))


embedding_vector = embeddings[0]

# min value
min_value = np.min(embedding_vector)

print(f'Embedding vector shape: {embedding_vector.shape}')
print(f'Minimum value in the embedding vector: {min_value}')

Embedding vector shape: (512,)
Minimum value in the embedding vector: -0.11726373885183883


### Cosine Similarity 

In [27]:
# Check the norm of the embedding vector
np.linalg.norm(embedding_vector)

np.float64(1.0)

#### Q2. Cosine Similarity with another vector


In [34]:
def get_embedding(text: str) -> np.ndarray:
    """    Get the embedding vector for a given text.

    Args:
        text (str): The input text to be embedded.  
    Returns:
        np.ndarray: The embedding vector for the input text.
    """
    embeddings = list(embedding_model.embed([text]))
    return embeddings[0]


def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """Calculate the cosine similarity between two vectors.
    Args:
        vec1 (np.ndarray): The first vector.
        vec2 (np.ndarray): The second vector.
    Returns:
        float: The cosine similarity between the two vectors.
    """
    return vec1.dot(vec2)


doc_1 = 'I just discovered the course. Can I join now?'
doc_2 = 'I just found out about the course. Is it too late to join?'

embedding_1 = get_embedding(doc_1)
embedding_2 = get_embedding(doc_2)

similarity = cosine_similarity(embedding_1, embedding_2)

print(f'Cosine similarity between the two documents: {similarity:.4f}')

Cosine similarity between the two documents: 0.9095


#### Q3. Ranking by Cosine

In [ ]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
'section': 'General course-related questions',
'question': 'Course - Can I still join the course after the start date?',
'course': 'data-engineering-zoomcamp'},
{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
'section': 'General course-related questions',
'question': 'Course - Can I follow the course after it finishes?',
'course': 'data-engineering-zoomcamp'},
{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
'section': 'General course-related questions',
'question': 'Course - When will the course start?',
'course': 'data-engineering-zoomcamp'},
{'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
'section': 'General course-related questions',
'question': 'Course - What can I do before the course starts?',
'course': 'data-engineering-zoomcamp'},
{'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
'section': 'General course-related questions',
'question': 'How can we contribute to the course?',
'course': 'data-engineering-zoomcamp'}]

In [ ]:
# Get the embedding for the query
query = 'Can I still join the course after the start date?'

q = get_embedding(query)

In [46]:
# getting the cosine similarity for each document (only text)
vector_embeddings = [get_embedding(doc['text']) for doc in documents]

V = np.array(vector_embeddings)

print(f'Embedding shape: {V.shape}')

Embedding shape: (5, 512)


In [ ]:
# Calculate the cosine similarity for each document with respect to the query
doc_ranks = V.dot(q)

# Get the index of the top document
top_1 = np.argmax(doc_ranks)

print(f'Top document index: {top_1}')

Top document index: 1


#### Q4. Ranking by cosine, version two

In [ ]:
full_texts = [] 

for doc in documents:
    full_texts.append(doc['question'] + ' ' + doc['text'])

In [48]:
full_texts

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin t

In [49]:
vector_embeddings = [get_embedding(text) for text in full_texts]

V = np.array(vector_embeddings)

print(f'Embedding shape: {V.shape}')

Embedding shape: (5, 512)


In [53]:
doc_ranks = V.dot(q)

print(f'Document ranks: {doc_ranks}\n')

top_1 = np.argmax(doc_ranks)

print(f'Top document index: {top_1}')

Document ranks: [0.92732523 0.85948475 0.83966021 0.76320298 0.76044493]

Top document index: 0


#### Q5. Selecting the embedding model

In [62]:
embedding_models = {}

for model in TextEmbedding.list_supported_models():
    # I want to find what is the smaller embedding dimension
    # loop through all the models and find the one with the smallest dimension
    if model['dim'] == 512:
        embedding_models[model['model']] = model['dim']
    if model['dim'] == 384:
        embedding_models[model['model']] = model['dim']
    if model['dim'] == 256:
        embedding_models[model['model']] = model['dim']
    if model['dim'] == 128:
        embedding_models[model['model']] = model['dim']

print(json.dumps(embedding_models, indent=2))

{
  "BAAI/bge-small-en": 384,
  "BAAI/bge-small-en-v1.5": 384,
  "BAAI/bge-small-zh-v1.5": 512,
  "snowflake/snowflake-arctic-embed-xs": 384,
  "snowflake/snowflake-arctic-embed-s": 384,
  "Qdrant/clip-ViT-B-32-text": 512,
  "sentence-transformers/all-MiniLM-L6-v2": 384,
  "jinaai/jina-embeddings-v2-small-en": 512,
  "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2": 384
}


384 is smaller dimensionality for model that are supported by `fastembed`

#### Q6. Indexing with qdrant

In [9]:
# Data preparation
with open('../documents.json', 'r') as f:
    documents_raw = json.load(f)

documents = []

for course in documents_raw: 
    course_name = course['course'] 
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(f'Number of documents: {len(documents)}')

Number of documents: 375


In [2]:
# Initialize Qdrant client
client = QdrantClient(url="http://localhost:6333") 

In [11]:
# Model Configuration
EMBEDDING_MODEL = "BAAI/bge-small-en"

EMBEDDING_DIM = 384

In [12]:
# collection name 
collection_name = "ml-zoomcamp-faq"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIM,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [14]:
# Creating Points to be inserted into the collection
points = []

for i, doc in enumerate(documents):
    # concatenate question and text for the full content
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=EMBEDDING_MODEL)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [16]:
points[:5]

[PointStruct(id=0, vector=Document(text='How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', model='BAAI/bge-small-en', options=None), payload={'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I 

In [22]:
# Upserting the points into the collection
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:  20%|██        | 1/5 [00:00<00:00,  5.71it/s]p:\Projects\LLM-ZoomCamp\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Karthik\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 files: 100%|██████████| 5/5 [0

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [23]:
query = "I just discovered the course. Can I join now?"

In [25]:
search_results = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query,
        model=EMBEDDING_MODEL
    ),
    limit=5,
    with_payload=True,
)

search_results.points

[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=6, version=0, score=0.86918855, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=No